## Count words in purpose/design/findings/originality

In [2]:
import os
import pandas as pd

# base_dir = '/home/ubuntu/efs/emerald/data/fulltext-abstract_sections/'
# wc = []
# for fname in os.listdir(base_dir):
#     if fname.endswith('.target') and 'bpe' not in fname:
#         with open(base_dir + fname) as f:
#             for line in f:
#                 wc.append(line.count(' ') + 1)
# wc_purpose, wc_method, wc_finding, wc_origin = zip(*[(wc[i*4], wc[i*4+1], wc[i*4+2], wc[i*4+3]) for i in range(len(wc)//4)])

# dfwc = pd.DataFrame({'word_count': wc})
# dfwc_sections = pd.DataFrame({'purpose': wc_purpose, 'method': wc_method, 'finding': wc_finding, 'origin': wc_origin, })
# dfwc_sections.describe()
# dfwc.describe()

### from emerald_with_structured_abstr.jsonl

In [3]:
import json
jsonl = []
with open('/home/ubuntu/efs/emerald/emerald_with_structured_abstr.jsonl') as f:
    for line in f:
        jsonl.append(json.loads(line.strip()))
df = pd.DataFrame({'jsonl': jsonl})

In [8]:
def fulltext_length(obj):
    text = ' '.join(i for i in obj['section_names'] if i != '__NO_TITLE__') + ' ' + ' '.join(par for sec in obj['sections'] for par in sec)
    return len(text.split())

df['fulltext_length'] = df.jsonl.apply(fulltext_length)

In [9]:
def abstract_length(j):
    abstract_sections_names = j['abstract_sections_names']
    abstract_sections = j['abstract_sections']

    section_length = {}
    for title, sec in zip(abstract_sections_names, abstract_sections):
        if title not in ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value'):
            continue
        text = ' '.join(par for par in sec)
        section_length[title] = len(text.split())

    return [section_length[i] for i in ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value')]

df['abstract_length'] = df.jsonl.apply(abstract_length)

In [10]:
df['ab_len_purpose'], df['ab_len_design'], df['ab_len_findings'], df['ab_len_origin'] = zip(*df.abstract_length)
df['abstract_length'] = df['abstract_length'].apply(sum)

In [11]:
df.describe(percentiles=[.01, .25, .5, .75, .99])

,fulltext_length,abstract_length,ab_len_purpose,ab_len_design,ab_len_findings,ab_len_origin
count,60146.000000,60146.000000,60146.000000,60146.000000,60146.000000,60146.000000
mean,5665.592325,195.905131,47.875254,45.300801,60.121504,42.607572
std,2231.319022,59.988423,25.308768,24.598482,28.951950,21.821588
min,55.000000,19.000000,6.000000,1.000000,2.000000,1.000000
1%,988.000000,82.000000,15.000000,9.000000,16.000000,11.000000
25%,4329.000000,157.000000,29.000000,28.000000,40.000000,27.000000
50%,5621.000000,190.000000,42.000000,41.000000,55.000000,38.000000
75%,6926.000000,228.000000,60.000000,58.000000,75.000000,53.000000
99%,11751.950000,377.000000,128.000000,121.000000,148.000000,112.000000
max,71313.000000,2191.000000,580.000000,681.000000,973.000000,377.000000


In [29]:
df[df.abstract_length < 50].shape

(39, 7)

In [38]:
j = df[df.abstract_length < 50]['jsonl'].tolist()[0]
print(j['journal'], '|||', j['title'])

International Journal of Sociology and Social Policy ||| Sport in Europe's era of austerity: crisis or adaptation?


In [43]:
from pprint import pprint
# pprint(j)

In [48]:
df[df.abstract_length > 500].shape

(83, 7)

In [42]:
j = df[df.abstract_length > 1000]['jsonl'].tolist()[0]
print(j['journal'], '|||', j['title'])

International Hospitality Review ||| Status and scope of project management in the hospitality industry


In [46]:
# pprint(j)

### Section titles in fulltext
* how many papers with intro section
* how many papers with method section
* ...

In [103]:
# ('Purpose', 'Design/methodology/approach', 'Findings', 'Originality/value')
def find_section_title_like(j, ts):
    section_names = j['section_names']
    for sn in section_names:
        for i in ts:
            if i.lower() in sn.lower():
                return 1
    return 0

# one-section-only source: 0. fulltext; 1. intro 2. method 3. results 4. conclusion
# leave-one-out
# target: purpose, method, finding, value
df['has_intro'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['intro', 'background']))
df['has_purpose'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['purpose']))
df['has_design'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['design', 'method', 'approach']))
df['has_result'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['result', 'find']))
df['has_origin'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['origin', 'value']))
df['has_conclu'] = df.jsonl.apply(lambda x: find_section_title_like(x, ['conclu']))

In [104]:
df.head(2)

,jsonl,fulltext_length,abstract_length,ab_len_purpose,ab_len_design,ab_len_findings,ab_len_origin,has_intro,has_design,has_result,has_origin,has_conclu,has_purpose
0,{'title': 'From the informal economy to the me...,6448,162,46,39,50,27,1,0,0,1,1,0
1,{'title': 'How close are formal and informal w...,5979,235,113,28,69,25,1,0,0,0,0,0


In [105]:
tmp = df[[i for i in df.columns if i.startswith('has_')]].agg(['sum'])
tmp

,has_intro,has_design,has_result,has_origin,has_conclu,has_purpose
sum,51691,40285,33872,1450,44797,641


In [106]:
# percent
tmp / df.shape[0]

,has_intro,has_design,has_result,has_origin,has_conclu,has_purpose
sum,0.859425,0.669787,0.563163,0.024108,0.744804,0.010657


In [101]:
# df.jsonl.apply(lambda x: find_section_title_like(x, ['introduction'])).sum()

ModuleNotFoundError: No module named 'transformers'